In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import requests
import dotenv
import os

dotenv.load_dotenv(".env")

True

Combining jul-sept and oct-dec data for specific facilities, then putting them into a dataframe

In [3]:
df=pd.DataFrame()

for facility_id in range(6,34):
  if facility_id == 11 or facility_id == 14:
    file = pd.read_json(f'data/carpark_history_6_months/facility_{facility_id}.json')
  
    df = pd.concat([df, file]).reset_index(drop=True)
    continue
  
  first_3_months_file_path = f'data/carpark_history_jul_sep/facility_{facility_id}.parquet'
  last_3_months_file_path = f'data/carpark_history_oct_dec/facility_{facility_id}.parquet'
  
  first_3_months_file = pd.read_parquet(first_3_months_file_path)
  last_3_months_file = pd.read_parquet(last_3_months_file_path)
  
  # Adding the facility data into one dataframe
  facility_6_months_data = pd.concat([first_3_months_file, last_3_months_file]).reset_index(drop=True)
  
    # Combining facility data with existing dataframe with all the data
  if facility_id == 6:
    df = facility_6_months_data
  else:
    df = pd.concat([df, facility_6_months_data]).reset_index(drop=True)

df

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id
0,207210,741448862,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:01:02,6,Gordon Henry St North Car Park,207210TPR001
1,207210,741450092,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:21:32,6,Gordon Henry St North Car Park,207210TPR001
2,207210,741468991,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:36:31,6,Gordon Henry St North Car Park,207210TPR001
3,207210,741469862,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:51:02,6,Gordon Henry St North Car Park,207210TPR001
4,207210,741474196,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T07:03:16,6,Gordon Henry St North Car Park,207210TPR001
...,...,...,...,...,...,...,...,...,...,...
2924540,2126158,757341730,384,"[{'occupancy': {'loop': '106782', 'monthlies':...",1,"{'loop': '106782', 'monthlies': None, 'open_ga...",2023-12-31T23:42:10,33,Cherrybrook Car Park,2126158TPR001
2924541,2126158,757341876,384,"[{'occupancy': {'loop': '106783', 'monthlies':...",1,"{'loop': '106783', 'monthlies': None, 'open_ga...",2023-12-31T23:44:36,33,Cherrybrook Car Park,2126158TPR001
2924542,2126158,757341939,384,"[{'occupancy': {'loop': '106784', 'monthlies':...",1,"{'loop': '106784', 'monthlies': None, 'open_ga...",2023-12-31T23:45:39,33,Cherrybrook Car Park,2126158TPR001
2924543,2126158,757342115,384,"[{'occupancy': {'loop': '106785', 'monthlies':...",1,"{'loop': '106785', 'monthlies': None, 'open_ga...",2023-12-31T23:48:35,33,Cherrybrook Car Park,2126158TPR001


In [4]:
df2 = df.copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924545 entries, 0 to 2924544
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   tsn                object
 1   time               object
 2   spots              object
 3   zones              object
 4   ParkID             object
 5   occupancy          object
 6   MessageDate        object
 7   facility_id        object
 8   facility_name      object
 9   tfnsw_facility_id  object
dtypes: object(10)
memory usage: 223.1+ MB


Save to file

In [5]:
list_to_change = ['time','spots','ParkID','facility_id']

for col in list_to_change:
  df2[col] = df2[col].astype(int)

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924545 entries, 0 to 2924544
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   tsn                object
 1   time               int32 
 2   spots              int32 
 3   zones              object
 4   ParkID             int32 
 5   occupancy          object
 6   MessageDate        object
 7   facility_id        int32 
 8   facility_name      object
 9   tfnsw_facility_id  object
dtypes: int32(4), object(6)
memory usage: 178.5+ MB


Giving missing TSN values to facilities 29,30,31

In [6]:
df_tsn_missing = df2.loc[df2['tsn'] == '']
df_tsn_missing

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id
2348544,,747791940,351,"[{'occupancy': {'loop': '12292', 'monthlies': ...",1,"{'loop': '12292', 'monthlies': None, 'open_gat...",2023-09-12T09:59:00,29,Kellyville North Car Park,
2348545,,747791945,351,"[{'occupancy': {'loop': '12292', 'monthlies': ...",1,"{'loop': '12292', 'monthlies': None, 'open_gat...",2023-09-12T09:59:05,29,Kellyville North Car Park,
2348546,,747792287,351,"[{'occupancy': {'loop': '12293', 'monthlies': ...",1,"{'loop': '12293', 'monthlies': None, 'open_gat...",2023-09-12T10:04:47,29,Kellyville North Car Park,
2348547,,747792382,351,"[{'occupancy': {'loop': '12294', 'monthlies': ...",1,"{'loop': '12294', 'monthlies': None, 'open_gat...",2023-09-12T10:06:22,29,Kellyville North Car Park,
2348548,,747792747,351,"[{'occupancy': {'loop': '12295', 'monthlies': ...",1,"{'loop': '12295', 'monthlies': None, 'open_gat...",2023-09-12T10:12:27,29,Kellyville North Car Park,
...,...,...,...,...,...,...,...,...,...,...
2569476,,747795400,777,"[{'occupancy': {'loop': '40584', 'monthlies': ...",1,"{'loop': '40584', 'monthlies': None, 'open_gat...",2023-09-12T10:56:40,31,Bella Vista Car Park,
2569477,,747795413,777,"[{'occupancy': {'loop': '40585', 'monthlies': ...",1,"{'loop': '40585', 'monthlies': None, 'open_gat...",2023-09-12T10:56:53,31,Bella Vista Car Park,
2569478,,747795607,777,"[{'occupancy': {'loop': '40586', 'monthlies': ...",1,"{'loop': '40586', 'monthlies': None, 'open_gat...",2023-09-12T11:00:07,31,Bella Vista Car Park,
2569479,,747795626,777,"[{'occupancy': {'loop': '40587', 'monthlies': ...",1,"{'loop': '40587', 'monthlies': None, 'open_gat...",2023-09-12T11:00:26,31,Bella Vista Car Park,


In [7]:
df_tsn_missing['facility_id'].unique()

array([29, 30, 31])

In [10]:
list(df_tsn_missing.index)

[2348544,
 2348545,
 2348546,
 2348547,
 2348548,
 2348549,
 2348550,
 2348551,
 2348552,
 2348553,
 2348554,
 2348555,
 2348556,
 2348557,
 2348558,
 2348559,
 2348560,
 2403950,
 2403951,
 2403952,
 2403953,
 2403954,
 2403955,
 2403956,
 2403957,
 2403958,
 2403959,
 2403960,
 2403961,
 2403962,
 2403963,
 2403964,
 2403965,
 2403966,
 2403967,
 2403968,
 2403969,
 2403970,
 2403971,
 2403972,
 2403973,
 2403974,
 2403975,
 2403976,
 2403977,
 2403978,
 2403979,
 2403980,
 2403981,
 2403982,
 2403983,
 2403984,
 2403985,
 2403986,
 2403987,
 2403988,
 2403989,
 2403990,
 2403991,
 2569435,
 2569436,
 2569437,
 2569438,
 2569439,
 2569440,
 2569441,
 2569442,
 2569443,
 2569444,
 2569445,
 2569446,
 2569447,
 2569448,
 2569449,
 2569450,
 2569451,
 2569452,
 2569453,
 2569454,
 2569455,
 2569456,
 2569457,
 2569458,
 2569459,
 2569460,
 2569461,
 2569462,
 2569463,
 2569464,
 2569465,
 2569466,
 2569467,
 2569468,
 2569469,
 2569470,
 2569471,
 2569472,
 2569473,
 2569474,
 2569475,


In [12]:
for index in list(df_tsn_missing.index):
  facility_id = df2.at[index,'facility_id']
  
  if facility_id == 29 or facility_id == 30:
    df2.at[index,'tsn'] = 2155382
  elif facility_id == 31:
    df2.at[index,'tsn'] = 2153478

df2.loc[df2['tsn'] == '']

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id


In [13]:
print(df2['tsn'].unique())
print(len(df2['tsn'].unique()))

['207210' 207210 '253330' 253330 '225040' 225040 '221210' 221210 '2101131'
 2101131 2101130 '2103108' 2103108 '2099207' 2099207 211420 '223210'
 223210 '217933' 217933 '217426' 217426 '276010' 276010 '256020' 256020
 '275010' 275010 '217010' 217010 '276220' 276220 '207720' 207720 '2155384'
 2155384 2155382 '2155382' 2153478 '2153478' '2154392' 2154392 '2126158'
 2126158]
44


In [14]:
df2.tsn = df2.tsn.astype(int)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924545 entries, 0 to 2924544
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   tsn                int32 
 1   time               int32 
 2   spots              int32 
 3   zones              object
 4   ParkID             int32 
 5   occupancy          object
 6   MessageDate        object
 7   facility_id        int32 
 8   facility_name      object
 9   tfnsw_facility_id  object
dtypes: int32(5), object(5)
memory usage: 167.3+ MB


# PREVIOUS WORK


Fetching data for July to September

In [ ]:
def date_getter(td):
    # Array that stores the dates to be searched for
    date_period_list = []

    # The last date to be searched for
    cutoff_date = datetime(2023, 9, 30)
    target_date = cutoff_date - td

    # Ensure that records of each day are obtained
    delta = timedelta(days=1)

    while target_date <= cutoff_date:
        date_period_list.append(target_date.strftime("%Y-%m-%d"))
        target_date += delta

    return date_period_list

def get_carpark_history(facility, dates_array):
    # Initialize data array
    data_array = []

    # Define the path for the JSON file
    json_file_path = f"./data/carpark_history_prev_3_months/facility_{facility}.json"

    # Set the request header
    headers = {
        "Authorization": f"apikey {os.environ.get('apikey')}"
    }

    # Delete the file if it exists
    if os.path.exists(json_file_path):
        os.remove(json_file_path)

    # Make a request for each date and aggregate the data
    for date in dates_array:
        url = f'https://api.transport.nsw.gov.au/v1/carpark/history?facility={facility}&eventdate={date}'
        response = requests.get(url, headers=headers).json()

        if data_array == []:
            data_array = response
        else:
            data_array = data_array + response

    # Save the data to a parquet file
    pd.DataFrame(data_array).to_parquet(json_file_path)

    # Give feedback on status
    print(f"Data for facility {facility} saved!")

In [ ]:
print("<---------- BEGINNING FETCHING DATA ---------->")

for facility_id in range(6,8):
  # restrict duplicate data
  if facility_id == 11 or facility_id == 14:
    print(f"<----- Skipping facility {facility_id} ----->")
    continue
  
  
  get_carpark_history(facility_id, date_getter(timedelta(days=91)))
  
print("<---------- COMPLETED FETCHING DATA ---------->")

In [ ]:
df_temp = pd.read_csv('data/weather/temperature.csv')
df_temp

In [ ]:
def date_time_getter(combined_date):
  date = combined_date.split(' ')[0]
  time = combined_date.split(' ')[1]
  return date, time

df_temp[['actual_date', 'actual_time']] = df_temp['date'].apply(lambda x: pd.Series(date_time_getter(x)))
# df_temp['actual_date'] = pd.to_datetime(df_temp['actual_date'])

df_temp

In [ ]:
df = df_temp.copy()

df.drop(columns='date',inplace=True)
df = df[['actual_date','actual_time','temperature']] 
df.rename(columns = {'actual_date':'date','actual_time':'time'}, inplace = True)
df.head()

In [ ]:
df.info()

In [ ]:
def aggregate_temperatures_per_hour(df):
    # Convert 'time' column to datetime object
    df['time'] = pd.to_datetime(df['time'],format="%H:%M:%S")
    
    # Extract hour component from 'time' column
    df['hour'] = df['time'].dt.hour
    
    # Group by 'date' and 'hour' and calculate mean temperature
    aggregated_df = df.groupby(['date', 'hour'])['temperature'].mean().reset_index()
    
    return aggregated_df
  
df_aggregate = aggregate_temperatures_per_hour(df)
df_aggregate

In [ ]:
cp = pd.read_json('data/carparks_original.json')
cp.head()

In [ ]:

coords_df = pd.read_json('data/coords.json')
coords_df.head()

In [ ]:
final_df = cp.copy()

final_df = final_df.merge(coords_df,on='facility_id',how='left')

final_df.sort_values(by='facility_id')

In [ ]:
df_final = pd.read_parquet('data/carpark_history_3_months_combined.parquet')
df_final.head(10)